In [ ]:
from azure_chat_openai_agent import AzureChatOpenAIAgent

agent = AzureChatOpenAIAgent
test_input = "What constitutes cyber security risk?"
response = agent.converse("", test_input, "")

# Print the response
print("Response from AzureChatOpenAIAgent:", response)

c:\Users\choonho.ooi\Hackathon\2025\SourceCode\commercial-contents-agent\hackathonenv\Lib\site-packages\langchain_core\vectorstores\base.py:1076: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='b56416db-6d5a-4627-af5a-95d8369a3e36', metadata={'has_checklist': False, 'has_manual': True, 'has_policy': True, 'has_report': True, 'has_safe_operating_procedure': False, 'has_template': True, 'path': 'CompliSpaceAssurance/Risk Register/Macro Risk Register.csv', 'price': 85.0, 'summary': 'This document contains risk descriptions, potential consequences, control policies, strategies, and responsible reporting measures for various organizational risks, including governance, compliance, workplace safety, and cyber security.', 'title': 'Risk Management Overview and Control Strategies'}, page_content='b\'Key,Title,Description of Risk,Potential Consequences,Control Policy,Risk Classification,Risk Flag,Master Risk Category,Reporting,Strategic Objectives,Person Responsible,Bus

Response from AzureChatOpenAIAgent: Cyber Security Risk refers to the failure to effectively implement policies, systems, and practices to identify risks to cyber security information assets and to prevent, detect, and respond to cyber security incidents. It is crucial to manage these risks effectively to avoid significant consequences.

### Potential Consequences of Cyber Security Risk:
1. Breach of law.
2. Data loss.
3. Systems loss.
4. Business disruption.
5. Civil litigation.
6. Reputational damage.

For more detailed strategies or control policies, you may need to consult a specific cyber security framework or policy document.

Let me know if you'd like further assistance!


In [ ]:
from chroma_knowledge_base_agent import ChromaKnowledgeBaseAgent

agent = ChromaKnowledgeBaseAgent
test_input = "What constitutes a cyber security risk?"
response = agent.retrieve(test_input)

print("Response from ChromaKnowledgeBaseAgent:", response)